# Text Repräsentationen
In diesem Teil des Labors wird es um eine Einfürung in Text Repräsentationen gehen. 

Wie Ihr vermutlich schon wisst, repräsentieren Computer Daten in Binärcode. Um Euch das Problem, das Computer mit Texten haben aufzuzeigen, hat euch der Computer eine Nachricht hinterlassen, die ihr Entschlüsseln müsst... 

In [1]:
%config Completer.use_jedi = False

In [2]:
binary_message = "01001001 00100000 01110011 01110000 01100101 01100001 01101011 00100000 01100010 01101001 01101110 01100001 01110010 01111001 00100000 01101111 01101110 01101100 01111001 00100001 00001010 01001001 00100000 01101100 01101001 01101011 01100101 00100000 01101110 01110101 01101101 01100010 01100101 01110010 01110011 00101100 00100000 01101110 01101111 01110100 00100000 01110100 01100101 01111000 01110100 00101110"

In [3]:
binary_chars = binary_message.split()
message = ""
for binary_char in binary_chars:
    binary_char_integer = int(binary_char, 2)
    character = chr(binary_char_integer)
    message += character
print(message)

I speak binary only!
I like numbers, not text.


Um Maschinen Text verständlich zu machen, müssen wir die Texte in eine andere Repräsentation bringen. Eine Kurze Einführung bieten die beiden folgenden Warm Up Aufgaben.

## Tokenization
Eine der ersten elementaren Schritte, um eine natürliche Sprache mit einem Computer zu verarbeiten ist das Aufteilen eines Fließtextes in kleinere Teile. Meist werden Texte direkt in sogenannte Tokens geteilt (Tokenization). Ein Token ist oftmals ein einzelnes Wort, es kann aber auch aus mehreren Wörtern bestehen, wenn diese zusammen für eine Entität stehen, z.B. die Stadt „New York“. Neben der Aufteilung in Tokens ist es auch möglich, einen Text zuerst in Sätze zu splitten (Sentence Tokenization). Welche Kombination der Methoden angewendet wird, hängt von der Aufgabe ab. 

### Warm Up 1: Tokenizer mit Regulären Ausdrücken

Die erste kleine Aufgabe besteht darin mit einem regulären Ausdruck ein Wort-Tokenizer zu bauen. (Diese Aufgabe sollte nicht mehr als 5 min. dauern) Der Tokenizer muss auch nicht perfekt sein. 

> Eine kurze Auffrischung über Reguläre Ausdrücke findet sich in https://regexr.com/

In [4]:
import re
text="Die Ente lacht und quakt."

tokenizer_regex=re.compile(r'[\s.]')
tokenizer_regex.split(text)

['Die', 'Ente', 'lacht', 'und', 'quakt', '']

Das Grundprinzip eines Tokenizers sollte euch nun klar sein. In der Praxis empfiehlt es sich allerdings, auf bestehende Tokenizer-Implementierungen zurückzugreifen.
Im Python-Umfeld ist https://spacy.io/api/tokenizer oder https://www.nltk.org/api/nltk.tokenize.html sehr empfehlenswert.

# Encoding

Der nächste Schritt, um mit Texten Maschinelles Lernen zu betreiben, ist das Umwandeln der Wörter in eine numerissche Repräsentation. In der nächsten Warm-Up Aufgabe wird es darum um Bag of Words gehen.

## Warm Up 2: Bag of Words mit scikit-learn
Ein Ansatz um Texte bzw. Tokens als Zahlen zu represäntieren ist ein Bag of Words. Hier wird jeder Text als Vektor dargestellt (Länge: Länge des Vokabulars). Jeder Eintrag im Vektor steht für ein Wort im Vokabular.

In der nächsten Aufgabe wollen wir mit dem vorgegebenen Textkorpus, den Satz **"Die Ente singt und quakt."** in einen bag-of-words-encoded Vector umwandeln.
Dazu bietet sich der `CountVectorizer` an (siehe [Dokumentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)).

> Kleiner Tipp: Der `CountVectorizer` übernimmt diesmal die Tokenisierung.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ["Die Ente lacht und quakt.",
          "Die Ente singt und tanzt."]
vectorizer = CountVectorizer()

abc =vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(abc.toarray())
#bag_of_words_encoded_sentence

['die', 'ente', 'lacht', 'quakt', 'singt', 'tanzt', 'und']
[[1 1 1 1 0 0 1]
 [1 1 0 0 1 1 1]]


Im  nächsten Schritt gebt die Länge der Satzrepräsentation aus. 

Wie verändert sich die Länge, wenn sich der Textkorpus vergrößert. Was sind die Vorteile und Nachteile von Bag of Words?

In [16]:
size_of_sentence_encoded= abc.shape[1]
print(size_of_sentence_encoded)

7


Vorteil: Leicht zu verstehen \
Nachteil: Mit steigendem Vokabular steigt auch die Dimension des zugehörigen Vektors. Context und Semantik geht durch Sortierung verloren.\
( Es ist spät. vs. ist es spät?)


## Word Embeddings

Natürliche Sprachen bieten uns mannigfaltige Möglichkeiten, dieselben Inhalte auf unterschiedliche Art und Weise auszudrücken. Wenn wir mit Texten arbeiten, reicht es daher in der Regel nicht aus, nur die Anzahl und das Vorkommen von bestimmten Wörtern in Texten zu betrachten, weil neben des bloßen Vorkommen eines Wortes auch dessen Anordnung und dessen Bedeutung eine Rolle spielt. Als zusätzliche und wichtige Ebene kommt hier also die Semantik ins Spiel.

Die Abbildung von Wörtern auf Vektoren erlaubt es uns, mit diesen zu rechnen und zum Beispiel Distanzen oder Ähnlichkeiten zu bestimmen. Embeddings haben den Vorteil, dass sie eine Dimensionsreduktion mit sich bringen und semantische Embeddings sorgen darüber hinaus dafür, dass "verwandte" Wörter einen geringen Abstand voneinander haben.

Wir wollen uns im Folgenden zunächst mit Word2Vec beschäftigen und eine einfache Version des CBOW-Ansatzes selbst implementieren.

Danach schauen wir uns Gensim als Bibliothek für Word-Embedding-Modelle an und werfen einen Blick auf Evaluationsmethoden für Embeddings sowie die ihnen inhärenten Biase.

## Aufgabe 1: Word2Vec CBOW
> You shall know a word by the company it keeps.
>
> -- <cite>J. R. Firth</cite>

Auf dem oben zitierten Prinzip beruhen die beiden als Word2Vec bekannt gewordenen Modelle CBOW und Skip Gram, die 2013 von [Tomas Mikolov et al.](https://arxiv.org/abs/1301.3781) bei Google entwickelt wurden.
Erstgenanntes Modell werden wir im Folgenden in einer einfachen Form selbst implementieren.

#### 1.1 Trainingsdaten
Die Beschaffung und Aufbereitung von Trainingsdaten ist ein wichtiger Schritt in jeder NLP-Pipeline. Jetzt drücken wir uns mal davor und greifen auf einen Datensatz zu, den wir schonmal vorarb für Euch vorbereitet haben. Wir haben uns entschieden Alice im Wunderland [Gutenberg Project Alice im Wunderland](https://www.gutenberg.org/cache/epub/19778/pg19778.txt) zu nutzen um Word Embeddings zu trainieren. Der Datensatz beinhaltet die Sätze aus aus der Geschichte. 

Der vorverarbeitete Datensatz ist als pickle abgespeichert und findet sich in [hier](https://drive.google.com/file/d/1RfCivF-wHf33S7TMxjpT78923ADXreMi/view). Wir werden den googledrivedownloader nutzen, um die Datei zu laden. Ladet den Datensatz mit Pickle als Testdatensatz aus.

In [17]:
#from google_drive_downloader import GoogleDriveDownloader as gdd

#gdd.download_file_from_google_drive(file_id='1RfCivF-wHf33S7TMxjpT78923ADXreMi',
#                                    dest_path='./download/alice_sentences.pkl',
#                                    unzip=False,
#                                    overwrite=True)

In [18]:
import pickle

with open(r"alice_sentences.pkl", "rb") as input_file:
    data_sample = pickle.load(input_file)

### 1.2 Datenvorbereitung
Wir haben einen tokenisierten Datensatz, der aus Listen von Wörtern besteht. Jede Liste repräsentiert einen Satz. Unser Modell soll aber hinterher mit Zahlen hantieren und zwar entweder mit Wortindizes, die jedes Wort im Vokabular über einen eindeutige Nummer referenzierbar machen, oder mit One-hot-Vektoren, die als Labels dienen, mit denen der tatsächliche Output des Modells verglichen werden kann.

In [21]:
from collections import OrderedDict

# Beim Mapping von Wörtern zu IDs und umgekehrt sollte eine reproduzierbare Reihenfolge sichergestellt werden,
# um das Modell später weitertrainieren und die Embedding-Matrix interpretieren zu können.
# Diese Datenstruktur kann, aber muss nicht, als Basis dienen.
unique_words = OrderedDict.fromkeys(# Liste alle Wörter, die in unserem data_sample vorkommen) 

# Mapping von Wort zu ID
word2id = # TODO

# Mapping von ID zu Wort
id2word = # TODO

# Unser Data Sample, aber mit IDs statt Wörtern 
# [['der', 'hund', 'der', 'bellt'], ['die', 'katz', 'miaut']] => [[0, 1, 0, 2], [3, 4, 5]]
numeric_docs = [[word2id[w] for w in doc] for doc in data_sample]

print('Word to id sample:', list(word2id.items())[:10], '\n')
print('Id to word sample:', list(id2word.items())[:10], '\n')
print('Documents as lists of integers:', numeric_docs[0][:10])


TypeError: unhashable type: 'list'

Wir halten einige wichtige Parameter für unser Modell fest. Die Größe des Kontextfensters sowie die Länge der Embeddingvektoren können nach Bedarf angepasst werden. Für unsere Demo wählen wir kleine Werte.

In [ ]:
vocabulary_size = # TODO
embedding_size = 50 # Länge der Embeddingvektoren
window_size = 2 # Größe des Kontextfensters. Wird nach rechts und links angewandt. Gesamter Kontext hier also 4 Wörter.

print('Vocabulary Size:', vocabulary_size)

### 1.3 Generator
Um nicht alle Trainingsdaten auf einmal im Speicher halten zu müssen, schreiben wir uns eine Generatorfunktion, die Batches einer frei wählbaren Größe zurückgibt. Unser Ansatz ist dennoch nicht völlig speicherschonend, weil wir uns die Datengrundlage für die Generierung dieser Batches, nämlich die Integerlisten in `numeric_docs` sehr wohl im Speicher vorhalten. Darüber sehen wir aber großzügig hinweg.

Die Generatorfunktion erzeugt zwei numpy-Arrays der Länge `batch_size`, von denen das eine Listen mit Indizes der Kontexwörter enthält, die der Embedding-Layer als Eingabe erwartet, und das andere die zugehörigen One-Hot-Encodings der Mittelwörter.

Fiktives und vereinfachtes Beispiel:
<pre><code>* Fenstergröße: 1
* Batch-Size: 2
* Wortindizes: 'die': 0, 'ente': 1, 'lacht': 2, 'und': '3, 'quakt': 4 (und damit Vokabulargröße 5)
* Korpus (Auszug): [['die', 'ente', 'lacht', 'und', 'quakt'], ...]


=> Rückgabe: [[0, 2], [1, 3]], [[0, 1, 0, 0, 0], [0, 0, 1, 0, 0]]
</code></pre>

In [ ]:
from keras.preprocessing import sequence
from keras.utils import np_utils
import numpy as np

def generate_context_word_batches(corpus, window_size, vocab_size, batch_size):
    X = []
    Y = []
    current_size = 0
    while True:
        # TODO: Here be dragons
        yield contexts, label_words # zwei numpy arrays


In [ ]:
# Schneller Test
test_batch_size = 3
test_window_size = 2

batch_gen = generate_context_word_batches(corpus=numeric_docs, window_size=test_window_size, vocab_size=vocabulary_size, batch_size=test_batch_size)
for i in range(0, 3): 
    x, y = next(batch_gen)
    for j in range(0, test_batch_size):
        print('Context (X):', [id2word[w] for w in x[j]], '-> Target (Y):', id2word[np.argwhere(y[j])[0][0]])  

### 1.4 Definition des Models
Als nächstes definieren wir unser Model. Dazu verwenden wir die Sequential API von Keras. Dieser [Blogpost](https://lilianweng.github.io/lil-log/2017/10/15/learning-word-embedding.html) bietet nochmal eine anschauliche Erklärung, wie word2vec CBOW funktioniert.
Das folgende Bild ist daraus und stellt die Architektur des Neuronalen Netzes dar:
<img src="https://lilianweng.github.io/lil-log/assets/images/word2vec-cbow.png" width=500 />

Weitere Informationen über den Embedding Layer finden sich [hier](https://keras.io/layers/embeddings/).

> **Tipp:** Nehmt euch wirklich Zeit Embeddings zu verstehen.

In [ ]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

#Modelldefinition
cbow = Sequential()
cbow.add(Embedding(input_dim=#TODO, output_dim=#TODO, input_length=#TODO))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(#TODO: #Units, activation=#TODO))
cbow.compile(loss=#TODO, optimizer='rmsprop')

# Zusammenfassung
print(cbow.summary())

### 1.5 Training
Jetzt wird es ernst: Wir trainieren unser Modell.
Da wir eine eigene Generatorfunktion verwenden, müssen wir `steps_per_epoch` angeben. Überlegt euch was damit genau gemeint ist, wofür uns das nützt und was wir bei der Berechnung beachten müssen. Tipp: das Modell sollte pro Epoche alle Trainingsdaten sehen. Überlegt auch ob die Cbow-Fenstergröße einen einfluss auf diese Anzahl hat.

Weil wir unser Modell gerne abspeichern möchten, zum Beispiel, um es später weiter zu trainieren, definieren wir eine Callback-Funktion, die das für uns übernimmt.

In [ ]:
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('embeddings.hd5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False)

In [ ]:
epochs = 5 #(kann gerne erhöht werden)
batch_size = 300
# das Modell sollte pro Epoche alle Trainingsdaten sehen. Überlegt auch ob die Cbow-Fenstergröße einen einfluss auf diese Anzahl hat.
steps_per_epoch = #TODO


cbow.fit_generator(# TODO, callbacks=[#TODO])    

### 1.6 Test
Nachdem wir unser Modell nur sehr kurz und nur auf wenigen Daten trainiert haben, ist davon auszugehen, dass die Ergebnisse nicht optimal sind. Einen kurzen Blick wollen wir dennoch riskieren.

Dazu extrahieren wir zunächst die Gewichte aus dem Embedding-Layer und schauen sie uns auszugsweise an.

In [ ]:
import pandas as pd
from keras.models import load_model

cbow = # TODO: Model laden
embedding_weights = # TODO: Auf Embedding Layer (1. Layer) des Modells zugreifen und dort die Gewichtsmatrix extrahieren

pd.DataFrame(# TODO: Was wollen wir anschauen?, index=list(id2word.values())).head()

Da durch scharfes Hinsehen nicht unmittelbar zu erkennen ist, wie gut unsere Embeddings schon sind, machen wir stichprobenartige Tests. Dazu wählen wir einige Wörter und berechnen für deren Embeddings die Ähnlichkeit mit allen anderen Embedding-Vektoren in unserer Gewichtsmatrix. Anschließend lassen wir uns die fünf ähnlichsten Wörter ausgeben.
Überlegt euch welches Distantzmaß für den Vergleich von Vektoren genutzt werden kann. Tipp: Die Cosinusähnlichkeit könnte damit was zu tun haben. 

In [ ]:
#Tipp: from sklearn.metrics.pairwise import ?

sample_terms = ['Alice', 'Hut', 'Kaninchen','Kaninchenbau']
sample_embeddings = # TODO

# Berechne die paarweisen Distanzen zwischen Beispielwörtern und Gesamtvokabular
distance_matrix = # TODO

In [ ]:
# Zeige die top fünf ähnlichsten Wörter zu unseren Beispielwörtern 
similar_words = {sample_term: [id2word[idx] for idx in distance_matrix[index].argsort()[1:6]] 
                   for index, sample_term in enumerate(sample_terms)}

similar_words

Seid ihr zufrieden mit den ähnlichen Wörtern? Woran kann es liegen, dass die Wörter nicht immer unbedingt Sinn ergeben?

## Aufgabe 2: Gensim als Wrapper für Word2Vec-Modelle
Embedding-Layer begegnen einem in der Praxis in der Tat häufig. In der Regel aber nicht als Bestandteile von reinen Word Embedding-Trainingsmodellen, sondern als erster Layer für Modelle mit anderen Aufgaben. Die Embeddings werden dann entweder mit vorberechneten Werten initalisiert und/oder werden im Training des Modells für die Downstream-Aufgabe (Textklassifikation, Übersetzung, ...) mittrainiert.

Eine komfortable Möglichkeit, eigene Word2Vec-Modelle zu trainieren, bietet [Gensim](https://radimrehurek.com/gensim/models/word2vec.html), eine Bibliothek, die für diese Modelle auch Wrapper bereitstellt, um komfortabler an die Embeddings zu kommen und mit diesen zu arbeiten.

Wir wollen uns im Folgenden einen kleinen Ausschnitt der Möglichkeiten, die Gensim bietet, anschauen.

### 2.0 Vorbereitung
Wir werden mit vortrainierten Google-News-Embeddings arbeiten, die ihr mit dem Code in den nächsten beiden Zellen herunterladen könnt. Falls das nicht funktionieren sollte, findet ihr die vortrainierten Embeddings [hier](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing). 

In [ ]:
!pip install googledrivedownloader

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1Fl11N_cX1RfJmTHV1C-RGIsjWeIZjbTN',
                                    dest_path='./download/GoogleNews-vectors-negative300.bin.gz',
                                    unzip=False,
                                    overwrite=True)

### 2.1 Word2Vec-Model laden
Die Vektoren haben eine Länge vin 300.
Zieht euch die Embeddings über den oben angegebenen Link und verwendet gensim, um sie anschließend zu laden.

**Hinweis**: Es kann einen Moment dauern, bis das Dictionary, das von Wort auf Embedding abbildet, erzeugt ist. Im Zweifel ein ```limit``` angeben und nur die ersten 1,5 Mio. Embeddings laden.

In [ ]:
!gunzip ./download/GoogleNews-vectors-negative300.bin.gz

In [ ]:
import gensim

embeddings = # TODO

Aus Spaß an der Freude können wir nun schauen, wie gut unser trainiertes Modell ist bzw. ob es bestimmte von Menschen wahrgenommene Analogien bestätigt. 

Die Analogien sind [hier](https://github.com/nicholas-leonard/word2vec/blob/master/questions-words.txt) zu finden.

In [ ]:
from gensim.test.utils import datapath

embeddings.evaluate_word_analogies(datapath("questions-words.txt"), restrict_vocab=30000)

### 2.2 Spaß mit Semantik
Im Folgenden wollen wir uns mit dem Mehrwert beschäftigen, den semantische Embeddings bieten. Für weitere Inspiration siehe zum Beispiel [hier](https://www.machinelearningplus.com/nlp/gensim-tutorial/) und die [Doku](https://radimrehurek.com/gensim/models/keyedvectors.html).

Mit semantischen Vektoren lassen sich zum Beispiel folgende Fragen beantworten:


In [ ]:
# Welche Stadt ist das New York Deutschlands? (Hinweis: 'New_York' ist als Token in den Embeddings enthalten)
print('Das deutsche New York ist: {}\n'.format(# TODO)))

# Was ist Emacs besonders ähnlich?
print('Ähnlichste Begriffe zu "Emacs": {}\n'.format(# TODO))

# Und wie sieht es mit Vim aus?
print('Ähnlichste Begriffe zu "Vim": {}\n'.format(# TODO))

# Wer ist eigentlich der Mozart der Naturwissenschaft?
print('Der Mozart der Naturwissenschaft ist: {}\n'.format(# TODO))

# Welches Wort verhält sich zu 'singing' wie 'burnt' zu 'burning'?
print('burning:burnt wie singing:{}\n'.format(# TODO))

# Sind sich Deutschland und Frankreich ähnlicher oder Deutschland und Kanada?
print('Ähnlichkeit DE, FR: {}'.format(# TODO))
print('Ähnlichkeit DE, CAN: {}'.format(# TODO))

Bei der Interpretation der Ergebnisse ist jedoch Vorsicht geboten: Es werden zwar semantische Beziehungen abgebildet, aber die entsprechen möglicherweise nicht immer den Erwartungen.

Wie ähnlich sind sich zum Beispiel "Leben" und "Tod", "kalt" und "warm", "Norden" und "Süden"?

Sind die Ergebnisse wie erwartet? Warum (nicht)?

In [ ]:
# TODO: Ähnlichkeiten berechnen

Die Embeddings sind nicht neutral, sondern spiegeln die Beziehungen wieder, die sich in den Trainingsdaten finden lassen.

In [ ]:
# Wird Wissenschaft von Frauen oder Männern gemacht?
print('Wissenschaft wird gemacht von: {}'.format(# TODO))
# Sind Mörder eher Schwarze, Weiße oder Asiaten?
print('Mörder sind: {}'.format(# TODO))
# Was bleibt vom Mann, wenn die Intelligenz abgezogen wird?
print('Mann ohne Intelligenz: {}'.format(# TODO))